In [1]:
import os, time, copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from PIL import Image

from utils import *
from dataset import *
from metrics import *
from FCN.fcn32 import *

In [2]:
set_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"] = '2'


In [3]:
# Load data
train_data = VOC(mode = 'train', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
train_loader = data.DataLoader(train_data, batch_size=1, num_workers = 0, shuffle=True)

val_data = VOC(mode = 'val', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
val_loader = data.DataLoader(val_data, batch_size=1, num_workers = 0, shuffle=True)

In [4]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0

    loss_sum = torch.zeros(1)
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    optimizer.zero_grad()
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
            
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc += pixel_accuracy(out_mask, target_mask)/20
        macc += mean_accuracy(out_mask, target_mask)/20
        mIU += mean_IU(out_mask, target_mask)/20
        fIU += frequency_weighted_IU(out_mask, target_mask)/20
        
        #calculate loss
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)/20
        loss.backward()
        loss_sum += loss
        
        
        # update gradient every 20 image
        if i%20 == 19 or i == 1111:
            optimizer.step()
            print("Batch %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i//20, loss_sum, pacc, macc, mIU, fIU))
            
            running_loss += loss_sum.item() * 20
            r_pacc += pacc * 20
            r_macc += macc * 20
            r_mIU += mIU * 20
            r_fIU += fIU * 20
            
            # batch initialize
            loss_sum, pacc, macc, mIU, fIU = 0, 0, 0, 0, 0
            optimizer.zero_grad()
            
            # show output mask
#             img = decode_segmap(out_mask)
#             plt.imshow(img); plt.axis('off'); plt.show()
        
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1112 for m in epoch_metric]
    
    print('Train:\t%s' %epoch_metric)
    return epoch_metric

In [5]:
def validate(model, device, val_loader, criterion, mode = 'val'):
    # switch to evaluate mode
    model.eval()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc = pixel_accuracy(out_mask, target_mask)
        macc = mean_accuracy(out_mask, target_mask)
        mIU = mean_IU(out_mask, target_mask)
        fIU = frequency_weighted_IU(out_mask, target_mask)
        
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)
        
        if i % 100 == 99:
            print("Val %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i, loss, pacc, macc, mIU, fIU))
        # statistics
        running_loss += loss.item()
        r_pacc += pacc
        r_macc += macc
        r_mIU += mIU
        r_fIU += fIU

    #epoch_loss = running_loss / 1111
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1111 for m in epoch_metric]
    
    if mode == 'val':
        print('Val metric:\t%s' %epoch_metric)
    else:
        print('Test metric:\t%s' %epoch_metric)
    
    return epoch_metric  #, copy.deepcopy(model.state_dict())

In [8]:
def train_model(model, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=50):
    
    criterion = nn.CrossEntropyLoss(ignore_index = 255)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5**(-4))

    if device == 'cuda':
        model.to(device)
        cudnn.benchmark = True
        criterion = criterion.cuda()
    
    since = time.time()
    M = {'train' : [], 'val' : []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        train_metric = train(model, device, train_loader, criterion, optimizer)
        M['train'].append(train_metric)
        
        # evaluate on validation set
        val_metric = validate(model, device, val_loader, criterion)
        M['val'].append(val_metric)
        
        if val_metric[1] > best_acc:
            best_acc = val_metric[1]
            best_model_wts = copy.deepcopy(model.state_dict())
        
        if epoch % 20 == 19:
            SaveDict(model.state_dict(), dict_name+'_epoch'+str(epoch+80))
        print()
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    print(model.load_state_dict(best_model_wts))
    return best_model_wts, M

In [ ]:
LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32 = FCN32()
fcn32_best, M = train_model(fcn32, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=200)

SaveDict(fcn32_best, 'fcn32_best')
SaveDict(M, 'fcn32_metric')

Epoch 0/199
----------
Batch 0)	3.037	0.000	0.000	0.000	0.000
Batch 1)	3.027	0.765	0.393	0.290	0.571
Batch 2)	3.015	0.745	0.454	0.344	0.576
Batch 3)	2.995	0.745	0.429	0.307	0.543
Batch 4)	2.970	0.755	0.452	0.340	0.579
Batch 5)	2.934	0.771	0.402	0.302	0.577
Batch 6)	2.915	0.741	0.438	0.309	0.534
Batch 7)	2.863	0.796	0.446	0.344	0.619
Batch 8)	2.835	0.717	0.410	0.284	0.518
Batch 9)	2.767	0.767	0.458	0.343	0.584
Batch 10)	2.723	0.775	0.408	0.310	0.595
Batch 11)	2.690	0.729	0.375	0.265	0.516
Batch 12)	2.666	0.702	0.429	0.299	0.528
Batch 13)	2.611	0.695	0.439	0.304	0.504
Batch 14)	2.528	0.733	0.442	0.313	0.528
Batch 15)	2.456	0.739	0.454	0.335	0.572
Batch 16)	2.455	0.687	0.413	0.282	0.480
Batch 17)	2.388	0.716	0.429	0.291	0.502
Batch 18)	2.294	0.772	0.442	0.340	0.602
Batch 19)	2.324	0.660	0.410	0.264	0.444
Batch 20)	2.092	0.757	0.408	0.308	0.574
Batch 21)	1.985	0.808	0.443	0.355	0.631
Batch 22)	1.953	0.744	0.413	0.298	0.557
Batch 23)	1.873	0.763	0.456	0.324	0.531
Batch 24)	1.724	0.815	0.463

Val 199)	0.812	0.839	0.500	0.389	0.604
Val 299)	1.738	0.336	0.575	0.195	0.165
Val 399)	0.447	0.945	0.500	0.463	0.859
Val 499)	0.541	0.896	0.500	0.441	0.779
Val 599)	1.768	0.638	0.500	0.309	0.382
Val 699)	0.325	0.946	0.500	0.467	0.871
Val 799)	3.704	0.106	0.500	0.051	0.010
Val 899)	0.245	0.974	0.500	0.479	0.916
Val 999)	0.423	0.940	0.500	0.459	0.844
Val 1099)	0.172	0.991	0.500	0.489	0.955
Val metric:	[1.072296688261062, 0.7507381707987197, 0.44155609430669884, 0.3223351647439094, 0.5564004086334556]

Epoch 3/199
----------
Batch 0)	0.923	0.779	0.467	0.355	0.618
Batch 1)	1.133	0.746	0.471	0.339	0.566
Batch 2)	1.173	0.715	0.452	0.322	0.521
Batch 3)	1.186	0.739	0.455	0.316	0.564
Batch 4)	1.034	0.735	0.469	0.336	0.523
Batch 5)	1.079	0.729	0.462	0.326	0.514
Batch 6)	1.076	0.735	0.374	0.269	0.559
Batch 7)	1.164	0.736	0.439	0.312	0.542
Batch 8)	0.988	0.765	0.410	0.297	0.557
Batch 9)	1.271	0.689	0.426	0.276	0.486
Batch 10)	1.002	0.746	0.465	0.343	0.555
Batch 11)	0.945	0.803	0.459	0.359	0.626
Ba

Batch 47)	0.919	0.758	0.471	0.367	0.587
Batch 48)	0.729	0.805	0.462	0.366	0.631
Batch 49)	0.981	0.742	0.447	0.331	0.551
Batch 50)	0.974	0.771	0.495	0.382	0.609
Batch 51)	0.879	0.770	0.484	0.370	0.591
Batch 52)	0.842	0.798	0.495	0.387	0.613
Batch 53)	0.742	0.804	0.522	0.422	0.634
Batch 54)	0.721	0.830	0.460	0.370	0.659
Batch 55)	0.537	0.469	0.335	0.256	0.327
Train:	[0.8809141493100914, 0.779797980504744, 0.4873035630950832, 0.3750917240799133, 0.5998818592491794]
Val 99)	0.415	0.895	0.500	0.438	0.767
Val 199)	0.970	0.764	0.333	0.265	0.591
Val 299)	0.332	0.962	0.333	0.281	0.713
Val 399)	0.998	0.752	0.500	0.362	0.523
Val 499)	0.346	0.860	0.500	0.422	0.711
Val 599)	0.172	0.970	0.500	0.477	0.912
Val 699)	0.819	0.802	0.333	0.254	0.581
Val 799)	0.996	0.722	0.578	0.518	0.616
Val 899)	1.983	0.461	0.616	0.222	0.186
Val 999)	0.906	0.842	0.772	0.709	0.764
Val 1099)	0.933	0.755	0.511	0.387	0.557
Val metric:	[0.8433309848597198, 0.7945402939931955, 0.516842350760337, 0.405446898203867, 0.62284917078

Batch 35)	0.643	0.808	0.508	0.409	0.642
Batch 36)	0.780	0.770	0.568	0.431	0.587
Batch 37)	0.852	0.738	0.533	0.406	0.560
Batch 38)	0.704	0.813	0.616	0.486	0.639
Batch 39)	0.641	0.824	0.541	0.437	0.661
Batch 40)	0.670	0.800	0.573	0.443	0.619
Batch 41)	0.482	0.864	0.510	0.439	0.735
Batch 42)	0.702	0.824	0.599	0.473	0.672
Batch 43)	0.516	0.865	0.559	0.471	0.713
Batch 44)	0.563	0.843	0.555	0.450	0.699
Batch 45)	0.712	0.796	0.521	0.413	0.608
Batch 46)	0.569	0.839	0.535	0.436	0.687
Batch 47)	0.676	0.811	0.564	0.457	0.649
Batch 48)	0.759	0.783	0.512	0.405	0.610
Batch 49)	0.699	0.790	0.601	0.456	0.610
Batch 50)	0.663	0.807	0.546	0.428	0.635
Batch 51)	0.470	0.873	0.492	0.414	0.718
Batch 52)	0.552	0.846	0.561	0.460	0.691
Batch 53)	0.510	0.853	0.560	0.476	0.717
Batch 54)	0.769	0.790	0.550	0.416	0.587
Batch 55)	0.335	0.505	0.352	0.288	0.417
Train:	[0.6591756116572044, 0.8123434517766143, 0.5579706105319711, 0.44523108309259346, 0.6476201920405564]
Val 99)	1.539	0.523	0.271	0.165	0.270
Val 199)	0.89

Batch 22)	0.729	0.778	0.607	0.467	0.624
Batch 23)	0.419	0.878	0.579	0.495	0.734
Batch 24)	0.507	0.836	0.645	0.530	0.699
Batch 25)	0.538	0.855	0.630	0.515	0.722
Batch 26)	0.508	0.848	0.599	0.486	0.691
Batch 27)	0.460	0.852	0.668	0.547	0.706
Batch 28)	0.537	0.837	0.577	0.469	0.685
Batch 29)	0.560	0.827	0.529	0.425	0.646
Batch 30)	0.531	0.846	0.525	0.435	0.706
Batch 31)	0.415	0.861	0.594	0.491	0.728
Batch 32)	0.393	0.878	0.626	0.528	0.760
Batch 33)	0.625	0.812	0.532	0.431	0.652
Batch 34)	0.624	0.804	0.586	0.485	0.616
Batch 35)	0.462	0.855	0.577	0.479	0.683
Batch 36)	0.614	0.805	0.616	0.491	0.607
Batch 37)	0.620	0.812	0.584	0.451	0.642
Batch 38)	0.686	0.795	0.587	0.467	0.630
Batch 39)	0.523	0.821	0.626	0.499	0.682
Batch 40)	0.559	0.842	0.586	0.484	0.685
Batch 41)	0.834	0.752	0.591	0.439	0.544
Batch 42)	0.505	0.844	0.673	0.544	0.714
Batch 43)	0.629	0.797	0.601	0.459	0.651
Batch 44)	0.623	0.820	0.567	0.452	0.652
Batch 45)	0.466	0.854	0.677	0.548	0.727
Batch 46)	0.647	0.791	0.626	0.480	0.622


...

Val 99)	0.117	0.937	0.802	0.671	0.862
Val 199)	0.213	0.917	0.951	0.731	0.847
Val 299)	0.254	0.910	0.757	0.511	0.692
Val 399)	0.433	0.855	0.449	0.383	0.705
Val 499)	0.072	0.976	0.965	0.794	0.877
Val 599)	0.218	0.904	0.923	0.789	0.780
Val 699)	0.415	0.871	0.472	0.334	0.642
Val 799)	0.376	0.849	0.722	0.559	0.652
Val 899)	0.119	0.947	0.949	0.843	0.858
Val 999)	0.392	0.882	0.569	0.415	0.652
Val 1099)	0.177	0.929	0.800	0.559	0.813
Val metric:	[0.23525554418537017, 0.9210703009650033, 0.7875019104183678, 0.6617433692555975, 0.7914226947984776]

Epoch 71/199
----------
Batch 0)	0.281	0.902	0.732	0.598	0.729
Batch 1)	0.295	0.893	0.721	0.605	0.744
Batch 2)	0.254	0.907	0.663	0.572	0.793
Batch 3)	0.278	0.898	0.723	0.609	0.745
Batch 4)	0.261	0.907	0.752	0.647	0.761
Batch 5)	0.310	0.890	0.834	0.681	0.683
Batch 6)	0.218	0.925	0.713	0.630	0.805
Batch 7)	0.217	0.925	0.814	0.721	0.819
Batch 8)	0.316	0.895	0.661	0.554	0.749
Batch 9)	0.300	0.892	0.716	0.592	0.749
Batch 10)	0.241	0.919	0.775	0.675	0.796
Ba

Batch 46)	0.246	0.919	0.724	0.640	0.814
Batch 47)	0.261	0.910	0.727	0.626	0.793
Batch 48)	0.282	0.907	0.739	0.648	0.784
Batch 49)	0.245	0.910	0.703	0.608	0.788
Batch 50)	0.285	0.898	0.714	0.615	0.769
Batch 51)	0.271	0.906	0.744	0.617	0.734
Batch 52)	0.216	0.925	0.703	0.613	0.817
Batch 53)	0.295	0.903	0.687	0.593	0.765
Batch 54)	0.284	0.900	0.723	0.622	0.761
Batch 55)	0.139	0.554	0.462	0.410	0.493
Train:	[0.2577357425642528, 0.9099993518006609, 0.7399614588470732, 0.6324596819505222, 0.773674215600142]
Val 99)	0.186	0.921	0.930	0.769	0.825
Val 199)	0.247	0.878	0.883	0.755	0.724
Val 299)	0.066	0.978	0.539	0.521	0.931
Val 399)	0.377	0.886	0.810	0.706	0.713
Val 499)	0.290	0.897	0.673	0.526	0.801
Val 599)	0.277	0.881	0.721	0.552	0.750
Val 699)	0.295	0.896	0.875	0.775	0.759
Val 799)	0.047	0.982	0.734	0.684	0.916
Val 899)	0.204	0.915	0.819	0.690	0.714
Val 999)	0.175	0.946	0.821	0.695	0.812
Val 1099)	0.157	0.970	0.974	0.849	0.905
Val metric:	[0.2421439321297552, 0.9192116452267055, 0.795002898

Batch 34)	0.212	0.926	0.792	0.687	0.814
Batch 35)	0.250	0.912	0.721	0.612	0.782
Batch 36)	0.188	0.934	0.810	0.687	0.795
Batch 37)	0.255	0.911	0.759	0.649	0.797
Batch 38)	0.231	0.918	0.750	0.647	0.788
Batch 39)	0.183	0.935	0.760	0.672	0.819
Batch 40)	0.227	0.922	0.700	0.604	0.815
Batch 41)	0.282	0.904	0.740	0.635	0.764
Batch 42)	0.253	0.913	0.654	0.562	0.773
Batch 43)	0.268	0.906	0.708	0.557	0.757
Batch 44)	0.254	0.913	0.752	0.635	0.792
Batch 45)	0.217	0.921	0.710	0.608	0.788
Batch 46)	0.323	0.885	0.724	0.610	0.752
Batch 47)	0.221	0.926	0.703	0.611	0.793
Batch 48)	0.237	0.916	0.791	0.688	0.800
Batch 49)	0.290	0.899	0.728	0.625	0.757
Batch 50)	0.253	0.910	0.736	0.646	0.792
Batch 51)	0.258	0.910	0.770	0.621	0.718
Batch 52)	0.189	0.942	0.735	0.628	0.781
Batch 53)	0.279	0.893	0.782	0.658	0.750
Batch 54)	0.250	0.914	0.786	0.667	0.787
Batch 55)	0.171	0.538	0.441	0.372	0.439
Train:	[0.2518008317021157, 0.9125118326007837, 0.7484547643288347, 0.6395783125752523, 0.7775667616367384]
Val 99)	0.13

Batch 21)	0.209	0.923	0.784	0.691	0.800
Batch 22)	0.211	0.922	0.760	0.663	0.802
Batch 23)	0.191	0.935	0.720	0.628	0.808
Batch 24)	0.256	0.919	0.743	0.642	0.784
Batch 25)	0.223	0.918	0.812	0.705	0.794
Batch 26)	0.178	0.939	0.788	0.694	0.834
Batch 27)	0.262	0.908	0.737	0.635	0.765
Batch 28)	0.252	0.919	0.741	0.645	0.804
Batch 29)	0.264	0.910	0.644	0.560	0.779
Batch 30)	0.237	0.915	0.760	0.608	0.700
Batch 31)	0.241	0.918	0.770	0.648	0.783
Batch 32)	0.245	0.916	0.722	0.615	0.750
Batch 33)	0.313	0.890	0.756	0.639	0.751
Batch 34)	0.291	0.897	0.777	0.657	0.736
Batch 35)	0.233	0.920	0.744	0.631	0.780
Batch 36)	0.210	0.922	0.777	0.663	0.829
Batch 37)	0.225	0.918	0.760	0.657	0.804
Batch 38)	0.313	0.886	0.780	0.650	0.732
Batch 39)	0.268	0.909	0.796	0.687	0.800
Batch 40)	0.345	0.878	0.679	0.576	0.733
Batch 41)	0.315	0.882	0.776	0.659	0.756
Batch 42)	0.241	0.916	0.811	0.685	0.769
Batch 43)	0.227	0.922	0.766	0.658	0.765
Batch 44)	0.220	0.917	0.754	0.618	0.761
Batch 45)	0.305	0.901	0.725	0.629	0.754


Batch 8)	0.182	0.935	0.751	0.634	0.800
Batch 9)	0.242	0.916	0.819	0.668	0.722
Batch 10)	0.216	0.924	0.779	0.684	0.820
Batch 11)	0.208	0.929	0.747	0.643	0.807
Batch 12)	0.332	0.882	0.710	0.592	0.737
Batch 13)	0.229	0.919	0.708	0.622	0.799
Batch 14)	0.209	0.929	0.728	0.642	0.822
Batch 15)	0.259	0.908	0.777	0.674	0.777
Batch 16)	0.229	0.922	0.702	0.610	0.792
Batch 17)	0.236	0.919	0.699	0.586	0.722
Batch 18)	0.241	0.913	0.705	0.609	0.778
Batch 19)	0.258	0.909	0.760	0.662	0.775
Batch 20)	0.221	0.924	0.772	0.680	0.809
Batch 21)	0.183	0.932	0.769	0.672	0.812
Batch 22)	0.210	0.921	0.824	0.717	0.810
Batch 23)	0.198	0.935	0.743	0.651	0.822
Batch 24)	0.149	0.948	0.850	0.752	0.800
Batch 25)	0.207	0.928	0.703	0.599	0.807
Batch 26)	0.226	0.922	0.799	0.689	0.794
Batch 27)	0.233	0.919	0.817	0.697	0.781
Batch 28)	0.222	0.919	0.743	0.639	0.809
Batch 29)	0.288	0.901	0.727	0.617	0.753
Batch 30)	0.256	0.913	0.735	0.629	0.780
Batch 31)	0.292	0.898	0.765	0.662	0.769
Batch 32)	0.240	0.916	0.738	0.638	0.783
Ba

In [9]:
# continue after 80 epoch because of the power loss 

LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32 = FCN32()
#Load dict
fcn32dict = LoadDict('fcn32_epoch79')
fcn32.load_state_dict(fcn32dict, strict = True)

fcn32_best, M = train_model(fcn32, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=120)

SaveDict(fcn32_best, 'fcn32_best')
SaveDict(M, 'fcn32_metric')

Epoch 0/119
----------
Batch 0)	0.223	0.931	0.721	0.638	0.820
Batch 1)	0.200	0.930	0.739	0.650	0.814
Batch 2)	0.308	0.892	0.774	0.665	0.744
Batch 3)	0.300	0.894	0.698	0.593	0.749
Batch 4)	0.260	0.912	0.641	0.567	0.783
Batch 5)	0.243	0.919	0.707	0.588	0.765
Batch 6)	0.227	0.924	0.800	0.689	0.802
Batch 7)	0.319	0.889	0.728	0.619	0.743
Batch 8)	0.225	0.915	0.831	0.724	0.810
Batch 9)	0.253	0.906	0.754	0.638	0.783
Batch 10)	0.194	0.933	0.812	0.698	0.773
Batch 11)	0.218	0.922	0.764	0.663	0.799
Batch 12)	0.268	0.904	0.737	0.628	0.773
Batch 13)	0.274	0.899	0.737	0.624	0.758
Batch 14)	0.272	0.906	0.811	0.674	0.751
Batch 15)	0.312	0.890	0.671	0.566	0.741
Batch 16)	0.231	0.925	0.727	0.621	0.810
Batch 17)	0.269	0.909	0.793	0.692	0.782
Batch 18)	0.241	0.920	0.676	0.591	0.801
Batch 19)	0.224	0.923	0.770	0.666	0.794
Batch 20)	0.205	0.931	0.758	0.661	0.814
Batch 21)	0.238	0.916	0.761	0.669	0.794
Batch 22)	0.208	0.932	0.765	0.663	0.794
Batch 23)	0.225	0.923	0.768	0.673	0.790
Batch 24)	0.277	0.897	0.723

Val 199)	0.150	0.948	0.950	0.882	0.881
Val 299)	0.191	0.951	0.926	0.832	0.850
Val 399)	0.085	0.968	0.968	0.838	0.898
Val 499)	0.252	0.901	0.875	0.790	0.792
Val 599)	0.289	0.909	0.600	0.475	0.642
Val 699)	0.471	0.755	0.628	0.441	0.681
Val 799)	0.159	0.944	0.945	0.836	0.854
Val 899)	0.108	0.964	0.949	0.888	0.868
Val 999)	0.096	0.972	0.625	0.595	0.900
Val 1099)	0.254	0.932	0.942	0.760	0.682
Val metric:	[0.21998570537534948, 0.9261500861818891, 0.7975866846196785, 0.6727046956687377, 0.7978238453087566]

Epoch 3/119
----------
Batch 0)	0.261	0.908	0.748	0.651	0.788
Batch 1)	0.311	0.887	0.753	0.632	0.742
Batch 2)	0.289	0.901	0.665	0.546	0.771
Batch 3)	0.252	0.912	0.754	0.645	0.780
Batch 4)	0.206	0.930	0.801	0.712	0.809
Batch 5)	0.254	0.907	0.695	0.588	0.771
Batch 6)	0.302	0.894	0.731	0.620	0.753
Batch 7)	0.225	0.922	0.746	0.637	0.794
Batch 8)	0.248	0.918	0.689	0.592	0.796
Batch 9)	0.259	0.911	0.715	0.608	0.774
Batch 10)	0.193	0.933	0.754	0.632	0.801
Batch 11)	0.263	0.911	0.734	0.645	0.778
B

Batch 47)	0.239	0.916	0.741	0.639	0.793
Batch 48)	0.201	0.931	0.764	0.655	0.825
Batch 49)	0.215	0.916	0.752	0.651	0.800
Batch 50)	0.272	0.910	0.773	0.666	0.793
Batch 51)	0.265	0.911	0.698	0.587	0.773
Batch 52)	0.197	0.933	0.780	0.681	0.816
Batch 53)	0.253	0.916	0.742	0.644	0.772
Batch 54)	0.288	0.899	0.753	0.650	0.761
Batch 55)	0.125	0.558	0.470	0.415	0.491
Train:	[0.2399822500058644, 0.9157895906374813, 0.753518630156978, 0.6467459002275736, 0.7818014711482723]
Val 99)	0.200	0.980	0.958	0.912	0.928
Val 199)	0.099	0.957	0.945	0.748	0.898
Val 299)	0.121	0.974	0.885	0.751	0.930
Val 399)	0.052	0.982	0.980	0.848	0.950
Val 499)	0.219	0.917	0.933	0.769	0.745
Val 599)	0.282	0.898	0.881	0.742	0.776
Val 699)	0.439	0.827	0.803	0.587	0.605
Val 799)	0.178	0.924	0.931	0.846	0.835
Val 899)	0.316	0.904	0.642	0.556	0.750
Val 999)	0.206	0.929	0.628	0.555	0.794
Val 1099)	0.115	0.980	0.500	0.486	0.943
Val metric:	[0.2162585562484005, 0.9271438895617251, 0.8012085746175971, 0.6740897592205816, 0.798863809

Batch 35)	0.225	0.921	0.778	0.677	0.808
Batch 36)	0.235	0.921	0.756	0.649	0.773
Batch 37)	0.205	0.931	0.784	0.653	0.786
Batch 38)	0.259	0.908	0.759	0.654	0.774
Batch 39)	0.208	0.928	0.699	0.612	0.825
Batch 40)	0.244	0.916	0.707	0.613	0.786
Batch 41)	0.229	0.920	0.766	0.658	0.814
Batch 42)	0.214	0.924	0.678	0.570	0.804
Batch 43)	0.314	0.891	0.771	0.659	0.750
Batch 44)	0.200	0.931	0.771	0.672	0.812
Batch 45)	0.192	0.931	0.785	0.664	0.761
Batch 46)	0.184	0.938	0.803	0.707	0.833
Batch 47)	0.191	0.934	0.720	0.624	0.814
Batch 48)	0.240	0.909	0.818	0.711	0.781
Batch 49)	0.262	0.910	0.783	0.655	0.745
Batch 50)	0.248	0.910	0.729	0.621	0.774
Batch 51)	0.282	0.894	0.781	0.670	0.745
Batch 52)	0.170	0.945	0.714	0.636	0.848
Batch 53)	0.199	0.937	0.759	0.657	0.798
Batch 54)	0.254	0.909	0.769	0.637	0.754
Batch 55)	0.168	0.538	0.415	0.356	0.454
Train:	[0.2365681470083676, 0.9170774773589945, 0.7567202765854015, 0.6487957744478231, 0.7833412106824565]
Val 99)	0.111	0.957	0.961	0.857	0.834
Val 199)	0.173

Batch 22)	0.216	0.929	0.751	0.659	0.807
Batch 23)	0.248	0.906	0.770	0.659	0.774
Batch 24)	0.196	0.927	0.832	0.719	0.800
Batch 25)	0.270	0.906	0.741	0.634	0.794
Batch 26)	0.222	0.924	0.694	0.599	0.799
Batch 27)	0.244	0.911	0.796	0.661	0.745
Batch 28)	0.184	0.935	0.750	0.632	0.778
Batch 29)	0.213	0.927	0.796	0.687	0.791
Batch 30)	0.185	0.937	0.705	0.614	0.821
Batch 31)	0.197	0.933	0.717	0.631	0.830
Batch 32)	0.262	0.910	0.773	0.669	0.764
Batch 33)	0.262	0.912	0.764	0.655	0.795
Batch 34)	0.189	0.932	0.780	0.665	0.791
Batch 35)	0.222	0.923	0.794	0.654	0.733
Batch 36)	0.235	0.916	0.785	0.668	0.744
Batch 37)	0.220	0.928	0.767	0.673	0.799
Batch 38)	0.203	0.929	0.781	0.678	0.817
Batch 39)	0.192	0.937	0.784	0.683	0.803
Batch 40)	0.256	0.911	0.716	0.624	0.779
Batch 41)	0.234	0.923	0.663	0.563	0.772
Batch 42)	0.261	0.905	0.711	0.604	0.777
Batch 43)	0.244	0.913	0.806	0.672	0.766
Batch 44)	0.205	0.933	0.819	0.713	0.818
Batch 45)	0.276	0.903	0.658	0.562	0.766
Batch 46)	0.222	0.919	0.777	0.676	0.793


...

Batch 16)	0.158	0.942	0.799	0.687	0.836
Batch 17)	0.160	0.943	0.800	0.698	0.831
Batch 18)	0.187	0.934	0.803	0.706	0.807
Batch 19)	0.134	0.954	0.848	0.755	0.843
Batch 20)	0.135	0.951	0.834	0.720	0.817
Batch 21)	0.162	0.945	0.851	0.759	0.837
Batch 22)	0.157	0.940	0.826	0.673	0.777
Batch 23)	0.172	0.940	0.781	0.689	0.825
Batch 24)	0.134	0.953	0.843	0.756	0.847
Batch 25)	0.165	0.941	0.785	0.683	0.826
Batch 26)	0.173	0.938	0.886	0.790	0.824
Batch 27)	0.151	0.943	0.796	0.706	0.817
Batch 28)	0.152	0.947	0.760	0.659	0.838
Batch 29)	0.170	0.940	0.763	0.665	0.827
Batch 30)	0.136	0.950	0.825	0.721	0.842
Batch 31)	0.122	0.957	0.837	0.746	0.836
Batch 32)	0.188	0.934	0.737	0.635	0.807
Batch 33)	0.206	0.928	0.763	0.658	0.791
Batch 34)	0.143	0.949	0.844	0.757	0.846
Batch 35)	0.194	0.929	0.806	0.692	0.808
Batch 36)	0.130	0.950	0.842	0.733	0.824
Batch 37)	0.204	0.925	0.808	0.690	0.797
Batch 38)	0.160	0.941	0.714	0.590	0.796
Batch 39)	0.206	0.926	0.773	0.669	0.779
Batch 40)	0.129	0.952	0.828	0.722	0.823


Batch 3)	0.176	0.935	0.759	0.662	0.813
Batch 4)	0.166	0.943	0.747	0.651	0.803
Batch 5)	0.187	0.929	0.777	0.669	0.799
Batch 6)	0.146	0.946	0.836	0.709	0.810
Batch 7)	0.187	0.932	0.832	0.696	0.751
Batch 8)	0.129	0.955	0.844	0.743	0.861
Batch 9)	0.161	0.942	0.835	0.746	0.836
Batch 10)	0.151	0.944	0.798	0.710	0.838
Batch 11)	0.203	0.926	0.776	0.666	0.779
Batch 12)	0.201	0.929	0.745	0.643	0.790
Batch 13)	0.190	0.932	0.751	0.653	0.822
Batch 14)	0.138	0.950	0.832	0.735	0.855
Batch 15)	0.209	0.921	0.846	0.724	0.773
Batch 16)	0.173	0.938	0.806	0.703	0.813
Batch 17)	0.163	0.941	0.775	0.678	0.829
Batch 18)	0.163	0.940	0.849	0.737	0.817
Batch 19)	0.161	0.941	0.792	0.672	0.817
Batch 20)	0.172	0.934	0.841	0.730	0.817
Batch 21)	0.167	0.939	0.847	0.700	0.761
Batch 22)	0.152	0.947	0.798	0.675	0.801
Batch 23)	0.159	0.941	0.839	0.756	0.840
Batch 24)	0.173	0.940	0.852	0.758	0.831
Batch 25)	0.170	0.938	0.765	0.644	0.787
Batch 26)	0.189	0.933	0.739	0.636	0.765
Batch 27)	0.170	0.943	0.785	0.695	0.843
Batch 2

Val 599)	0.413	0.840	0.412	0.327	0.657
Val 699)	0.106	0.967	0.971	0.795	0.898
Val 799)	0.094	0.970	0.912	0.780	0.918
Val 899)	0.095	0.980	0.985	0.927	0.930
Val 999)	0.096	0.966	0.931	0.736	0.869
Val 1099)	0.177	0.934	0.874	0.743	0.819
Val metric:	[0.1553116687271714, 0.944959007198888, 0.8575963254372653, 0.7192415323030918, 0.8247949884420382]

Epoch 116/119
----------
Batch 0)	0.199	0.928	0.709	0.607	0.797
Batch 1)	0.154	0.941	0.829	0.728	0.831
Batch 2)	0.169	0.939	0.866	0.760	0.820
Batch 3)	0.156	0.942	0.835	0.723	0.830
Batch 4)	0.190	0.930	0.722	0.623	0.795
Batch 5)	0.176	0.934	0.850	0.743	0.812
Batch 6)	0.180	0.930	0.806	0.702	0.800
Batch 7)	0.177	0.935	0.800	0.653	0.751
Batch 8)	0.139	0.947	0.830	0.726	0.847
Batch 9)	0.176	0.938	0.798	0.674	0.783
Batch 10)	0.119	0.960	0.842	0.749	0.861
Batch 11)	0.168	0.938	0.808	0.715	0.818
Batch 12)	0.138	0.953	0.882	0.763	0.830
Batch 13)	0.164	0.940	0.864	0.751	0.813
Batch 14)	0.186	0.932	0.815	0.706	0.777
Batch 15)	0.161	0.941	0.805	0.706	0.8

Batch 51)	0.209	0.924	0.833	0.700	0.749
Batch 52)	0.152	0.945	0.806	0.721	0.841
Batch 53)	0.173	0.943	0.803	0.685	0.791
Batch 54)	0.165	0.937	0.873	0.772	0.821
Batch 55)	0.096	0.565	0.502	0.445	0.491
Train:	[0.16304552179875134, 0.941137925687037, 0.8098028375123785, 0.7055710727354856, 0.8163123986759427]
Val 99)	0.124	0.952	0.956	0.846	0.851
Val 199)	0.162	0.941	0.929	0.813	0.808
Val 299)	0.163	0.940	0.952	0.842	0.843
Val 399)	0.067	0.976	0.490	0.376	0.889
Val 499)	0.074	0.967	0.961	0.855	0.877
Val 599)	0.215	0.924	0.859	0.676	0.747
Val 699)	0.205	0.942	0.955	0.686	0.545
Val 799)	0.128	0.965	0.564	0.406	0.830
Val 899)	0.194	0.921	0.916	0.748	0.741
Val 999)	0.098	0.961	0.971	0.872	0.887
Val 1099)	0.089	0.975	0.914	0.760	0.934
Val metric:	[0.14537869024062805, 0.9489382074269955, 0.8551865415888753, 0.7261259115965312, 0.8292456221749829]

Epoch 119/119
----------
Batch 0)	0.201	0.924	0.751	0.610	0.689
Batch 1)	0.142	0.949	0.806	0.709	0.836
Batch 2)	0.234	0.916	0.728	0.629	0.773
Batch 

0

In [11]:
len(M['train'])

120